# Fitting

Taking some material from https://github.com/klieret/HEPFittingTutorial/

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize, curve_fit

## Fit a line to data by minimizing the squared distance

The basic idea of fitting consists of minimizing a cost function by adjusting parameters of some function template.

Let's define some random data:

In [ ]:
x_data = np.linspace(-1, 1, 10)
y_data = -1 + 3 * x_data + 2 * np.random.random_sample(len(x_data))

And take a quick look at it:

In [ ]:
plt.plot(x_data, y_data, 'ko')

This looks like a line, so this is what we want to fit:

In [ ]:
def line(x, params):
    a, b = params
    return a * x + b

The line is defined as $f(\vec x) = a \vec x + b$ and maps a vector of x coordinates to y coordinates.
The two parameters a and b are collected in vector ``params``.

Now the idea is to minimize the distance of our function ``line`` to the y coordinates of the data, so we define
another function ``chi2``, which, for every set of parameters returns the sum of the squared distances of data points to function values:

In [ ]:
def chi2(params):
    return np.sum(np.square(y_data - line(x_data, params)))

Let's look at this step by step: 

* ``line(x_data, params)``: Here we passed on the parameters of ``chi2`` to the line function which we evaluate for all the data x values. The result is a vector y values.
* ``y_data - line(x_data, params)``: This is then the vector of distances between the data y values and the y values of our function
* ``np.square(y_data - line(x_data, params))``: The vector of squared distances
* ``np.sum(np.square(y_data - line(x_data, params)))``: Summing everything up

First, try to manually tune the parameters and see how `chi2` changes:

In [ ]:
params = [3, 0.1]
plt.plot(x_data, y_data, 'ko', label="data")
plt.plot(x_data, line(x_data, params), label="line")
plt.legend()
print("Chi2: ", chi2(params))

Of course we don't do this manually in practice - especially with higher number of parameters this can be challenging.

Scipy provides [`scipy.optimize.minimize`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html). It needs start points, which we just set to ``(0, 1)``

In [ ]:
result = minimize(chi2, (0, 1))

Note how ``minimize`` is a higher order function, that takes a function as first argument!

The results object contains quite a lot of useful information, but we just want the values of our parameters:

The value of `chi2` for the parameters that minimize it is:

In [ ]:
chi2(result.x)

Let's plot to see how well it fits:

In [ ]:
# Plotting our data point
plt.plot(x_data, y_data, 'ko', label="data")
plt.plot(x_data, line(x_data, result.x), label="fit")
plt.legend()

What we just did is called a "Least Square Fit"

Fitting an arbitrary (non-linear) Function to  weighted points (= measurements with uncertainties).

**Basic principle:**  
Minimize $\chi^2$, the quadratic difference between measurement points and fit, weighted by inverse uncertainty squared:

$$ \chi^2 = \sum \frac{(y_{meas}-y_{fit})^2}{  ( \Delta y )^2} $$

The resulting value for $\chi^2$ is an important check whether the fitting model is sensible:

$$ \left< \frac{\chi^2}{  (n_{points} - n_{par} )} \right> \approx 1$$

$n_{points} - n_{par}$ is the number of degrees of freedom (ndf) in our optimization problem.

**Note** In our example with the line we didn't divide by the uncertainty in the definition of `chi2`, so we made the implicit assumption that all data points are equally weighted. Not knowing the uncertainty of the data points also means we can't use $\chi^2$ to test goodness of fit by comparing it to the number of degrees of freedom.

## Using `scipy.optimize.curve_fit`

For a least-square-fit we don't need to write the cost function manually, but can instead use `scipy.optimize.curve_fit`.

For curve fit we don't need to put all parameters into one tuple. Instead the first argument of the fit function is interpreted to be the input data, and all remaining arguments the parameters:

In [ ]:
def f(x, a, b):
    return line(x, [a, b])

`curve_fit` will return the fitted parameters and their covariance matrix

In [ ]:
pfit, pcov = curve_fit(f, x_data, y_data, p0=(1, 0))
pfit, pcov

### Fit uncertainties and covariances

The covariance matrix shows the variances on the fit parameters on the diagonal and their covariances on the off diagonal elements.

**Note:** Since we did not provide uncertainties on the data points (would be done using the `sigma` argument to `curve_fit`) the overall normalization of $\chi^2$ is undetermined. By default, `curve_fit` multiplies the covariance matrix by $\chi^2_\mathrm{min}/\mathrm{ndf}$ which effectively scales the uncertainties on the data points to match the observed residuals after the fit. This can be turned off by using `absolute_sigma=True` which can be used when uncertainties are provided.

So the one standard deviation errors on the fit parameters will be given by

In [ ]:
a, b = pfit
a_err, b_err = np.sqrt(np.diag(pcov))
print("a = {:.2f} +/- {:.2f}".format(a, a_err))
print("b = {:.2f} +/- {:.2f}".format(b, b_err))

The covariances indicate how much the parameters are correlated, that is how much greater values of one parameter correspond to greater values of the other one.

You can try this out. Change on of the parameters slightly and see if there is a way to get a better `chi2` again by changing the other one:

In [ ]:
a_test = a + 1
(b_test,), _ = curve_fit(lambda x, b: f(x, a + 1, b), x_data, y_data, p0=(0))

In [ ]:
a_test, b_test = [a + 1, b]
plt.plot(x_data, y_data, "ko")
yfit = f(x_data, a_test, b_test)
plt.plot(x_data, yfit)
print("Chi2:", ((yfit - y_data) ** 2).sum())

Also, if we fix a to a value slightly away from the optimum, and fit `b` again, the optimal `b` won't change significantly:

In [ ]:
curve_fit(lambda x, b: f(x, a + 1, b), x_data, y_data, p0=(0))

In this case, we have the same number of points on the positive and negative x-axis, so a higher slope (parameter `a`) puts the data points on the positive x-axis below the fitted line and on the negative x-axis above the fitted line.

Changing the parameter `b` will cause an overall shift to the top or bottom, so there is no way to compensate a less optimal value of `a`.

The situation changes if we only look at data points on the positive x-axis:

In [ ]:
x_data2 = np.linspace(0, 1, 10)
y_data2 = -1 + 3 * x_data2 + np.random.random_sample(len(x_data2))

In [ ]:
plt.plot(x_data2, y_data2, "ko")

In [ ]:
pfit2, pcov2 = curve_fit(f, x_data2, y_data2, p0=(1, 0))
pfit2, pcov2

Here, the off diagonal elements have a significant (negative) non-zero value.

In [ ]:
a2, b2 = pfit2

Now we can compensate a slightly higher value of `a` by a slightly lower value of `b`:

In [ ]:
a2_test, b2_test = [a2 + 1, b2]
plt.plot(x_data2, y_data2, "ko")
yfit = f(x_data2, a2_test, b2_test)
plt.plot(x_data2, yfit)
print("Chi2:", ((yfit - y_data2) ** 2).sum())

In [ ]:
curve_fit(lambda x, b: f(x, a2 + 1, b), x_data2, y_data2, p0=(0))

Since we know the "real" distribution of our data points (we generated them after all) we can check with toy experiments how well the estimation of uncertainties and covariances from the fit worked:

In [ ]:
def fit_toy():
    x_data = np.linspace(0, 1, 10)
    y_data = -1 + 3 * x_data + np.random.random_sample(len(x_data))
    pfit, pcov = curve_fit(f, x_data, y_data, p0=(1, 0))
    return pfit, pcov

In [ ]:
toy_params = np.array([fit_toy()[0] for i in range(1000)]).T

In [ ]:
plt.scatter(*toy_params, marker=".")

In [ ]:
np.cov(toy_params)

In [ ]:
pcov2

Which looks pretty similar! Note, however, these intervals are typically interpreted as confidence intervals, so what we would actually be interested in is, do 68.27% of all intervals (in repeated experiments that assume different true values) contain the true value ([coverage probability](https://en.wikipedia.org/wiki/Coverage_probability)).

### Error propagation

One thing we can do with this covariance matrix is to visualize uncertainties on the fit using [linear error propagation](https://en.wikipedia.org/wiki/Propagation_of_uncertainty). In this case we can simply calculate it manually:

$$\sigma_y ^ 2 = \left(\frac{\partial y}{\partial a} \sigma_a \right)^2 + \left(\frac{\partial y}{\partial b} \sigma_b \right)^2 + 2\frac{\partial y}{\partial a}\frac{\partial y}{\partial b}\sigma_{ab}$$

where $\sigma_a, \sigma_b$ are the variances and $\sigma_{ab}$ is the covariance. With $y = ax + b$ this becomes:

$$\sigma_y ^ 2 = (x\sigma_a)^2 + \sigma_b^2 + 2x\sigma_{ab}$$

where we can take $\sigma_a, \sigma_b, \sigma_{ab}$ from the covariance matrix:

$$
\pmatrix{
    \sigma_a^2 & \sigma_{ab} \\
    \sigma_{ba} & \sigma_b^2
}
$$

Let's visualize it:

In [ ]:
sigma_y = np.sqrt((x_data2 * np.sqrt(pcov2[0, 0])) ** 2 + pcov2[1, 1] + 2 * x_data2 * pcov2[0, 1])

In [ ]:
y = line(x_data2, pfit2)
plt.fill_between(x_data2, y - sigma_y, y + sigma_y, alpha=0.5)
plt.plot(x_data2, y)
plt.plot(x_data2, y_data2, "ko")

For more generic templates/functions you can do that automatically. Either use the [uncertainties](https://pythonhosted.org/uncertainties) package (for functions described by simple formulas) or calculate it numerically by varying each parameter up and down and using half of the resulting interval as a replacement for $\frac{\partial f}{\partial x_i}\sigma_{x_i}$.

### Goodness of fit

Taking some inspiration from http://www.pp.rhul.ac.uk/~cowan/stat_course.html

Let's have a look at these data points (this time with uncertainties):

In [ ]:
x_data = np.array([1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0])
y_data = np.array([2.7, 3.9, 5.5, 5.8, 6.5, 6.3, 6.7, 6.2, 6.0])
yerr_data = np.array([0.3, 0.5, 0.7, 0.6, 0.4, 0.3, 0.7, 0.8, 0.5])

In [ ]:
plt.errorbar(x_data, y_data, yerr=yerr_data, fmt="ko")

And fit a line again:

In [ ]:
def f_linear(x, a, b):
    return a * x + b

In [ ]:
pfit, pcov = curve_fit(f_linear, x_data, y_data, sigma=yerr_data, absolute_sigma=True)

In [ ]:
plt.errorbar(x_data, y_data, yerr=yerr_data, fmt="ko")
plt.plot(x_data, f_linear(x_data, *pfit))

That doesn't look very great. How can we quantify the quality of this fit? We look at our $\chi^2$ statistic:

In [ ]:
def f_chi2(f, params, x, y, yerr):
    return (((f(x, *params) - y) / yerr) ** 2).sum()

In [ ]:
f_chi2(f_linear, pfit, x_data, y_data, yerr_data)

Reminder: As a rule of thumb, the number of degrees of freedom (number of data points - number of parameters) should be roughly equal to the $\chi^2$ statistic.

In [ ]:
len(x_data) - len(pfit)

So this rule of thumb already indicates this is not a very nice fit. We can be even more quantitative. This rule comes from the fact that the $\chi^2$ statistic actually follows a [Chi-squared distribution](https://en.wikipedia.org/wiki/Chi-squared_distribution) (which has ndf as expectation value) if we assume the data points follow a normal distribution. Using this we can calculate a p-value that answers the question "how often would we get such a high value of $\chi^2$ in repeated experiments, given that our function describes the data".

Scipy provides functions for common probability density functions among which there is also the chi-squared distribution. What we want to calculate is

$$p = \int\limits_{\chi^2_\mathrm{min}}^{\infty}f(\chi^2, \mathrm{ndf})\mathrm{d}\chi^2 = 1 - F(\chi^2_\mathrm{min}, \mathrm{ndf})$$

where $F(\chi^2_\mathrm{min}, \mathrm{ndf})$ is the cumulative distribution function of a chi-squared distribution which we can calculate using `scipy.stats.chi2.cdf`:

In [ ]:
import scipy.stats

In [ ]:
def chi2_pvalue(chi2, ndf):
    return 1 - scipy.stats.chi2.cdf(chi2, ndf)

In [ ]:
chi2_pvalue(
    f_chi2(f_linear, pfit, x_data, y_data, yerr_data),
    len(x_data) - len(pfit)
)

which is rather low, again indicating a bad fit!

<div class="alert alert-block alert-success">
    <b>Exercise</b> Fit a quadratic function to the data. What is $\chi^2 / \mathrm{ndf}$ now? What is the p-value?
</div>